In [1]:
# Import necessary libraries

!pip install ultralytics -q
# !pip install gradio -q
# !pip install pandas -q
!pip install deepface -q

In [2]:
# Import necessary libraries
from ultralytics import YOLO
from ultralytics.engine.results import Results  
from deepface import DeepFace
from PIL import Image
# import gradio as gr
import shutil
import pandas
import cv2
import os

In [3]:
def faceRecognition(input_image):
    # Path to the directory containing cropped objects
    cropped_objects_dir = "./faces/"
    
    # Path to the directory to save unknown faces
    unknown_faces_dir = "./unknown/"
    
    # Path to the directory to save known faces
    known_faces_dir = "./known/"
    
    # Initialize a list to store the extracted names
    extracted_names = []
    
    # Check if the 'unknown' folder exists, otherwise create it
    if not os.path.exists(unknown_faces_dir):
        os.makedirs(unknown_faces_dir)
    else:
        # If the 'unknown' folder exists, clear all files and subfolders
        for file_or_folder in os.listdir(unknown_faces_dir):
            file_or_folder_path = os.path.join(unknown_faces_dir, file_or_folder)
            if os.path.isfile(file_or_folder_path):
                os.remove(file_or_folder_path)
            elif os.path.isdir(file_or_folder_path):
                shutil.rmtree(file_or_folder_path)

    # Check if the 'known' folder exists, otherwise create it
    if not os.path.exists(known_faces_dir):
        os.makedirs(known_faces_dir)
    else:
        # If the 'known' folder exists, clear all files and subfolders
        for file_or_folder in os.listdir(known_faces_dir):
            file_or_folder_path = os.path.join(known_faces_dir, file_or_folder)
            if os.path.isfile(file_or_folder_path):
                os.remove(file_or_folder_path)
            elif os.path.isdir(file_or_folder_path):
                shutil.rmtree(file_or_folder_path)

    # Iterate through the image files in the directory
    for filename in os.listdir(cropped_objects_dir):
        if filename.endswith(".jpg"):
            img_path = os.path.join(cropped_objects_dir, filename)
            model = DeepFace.find(img_path=img_path, db_path="database", enforce_detection=False, model_name="Facenet512")

            # Check if a face was recognized in the image
            if model and len(model[0]['identity']) > 0:
                # Extract the name and append it to the list
                identity_path = model[0]['identity'][0]
                # Replace backslashes with forward slashes for cross-platform compatibility
                name = identity_path.replace('\\', '/').split('/')[1]
                
                # Save the known face into the 'known' folder
                known_faces_path = os.path.join(known_faces_dir, f"{len(extracted_names) + 1}_{name}.jpg")
                shutil.copy(img_path, known_faces_path)
                
            else:
                # If no face is recognized, set name to 'unknown'
                name = 'unknown'
                
                # Save the unknown face into the 'unknown' folder
                unknown_faces_path = os.path.join(unknown_faces_dir, f"{len(extracted_names) + 1}.jpg")
                shutil.copy(img_path, unknown_faces_path)
                
            extracted_names.append(name)
            
    return extracted_names

In [4]:
def faceExtraction(input_image, model, results):
    # Load the image
    image = Image.open(input_image)
    detected_objects = []

    if hasattr(results, 'boxes') and hasattr(results, 'names'):
        for box in results.boxes.xyxy:
            object_id = int(box[-1])
            object_name = results.names.get(object_id)
            x1, y1, x2, y2 = int(box[0]), int(box[1]), int(box[2]), int(box[3])

            detected_objects.append((object_name, (x1, y1, x2, y2)))

    # Create or clear the 'faces' directory
    if os.path.exists("faces"):
        shutil.rmtree("faces")
    os.makedirs("faces")

    # Crop and save each detected object
    for i, (object_name, (x1, y1, x2, y2)) in enumerate(detected_objects):
        object_image = image.crop((x1, y1, x2, y2))
        object_image.save(f"faces/face{i}.jpg")
        
    return 0

In [5]:
def faceDetection(input_image):
    model = YOLO('best.pt')
    results: Results = model.predict(input_image)[0]

    return faceExtraction(input_image, model, results)

In [6]:
image = input("enter image path : ")
faceDetection(image)
names = faceRecognition(image)
print(names)

enter image path :  World-Leaders.jpg


WARNING best.pt appears to require 'omegaconf', which is not in Ultralytics requirements.
AutoInstall will run now for 'omegaconf' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official Ultralytics model, i.e. 'yolo predict model=yolo11n.pt'
requirements: Ultralytics requirement ['omegaconf'] not found, attempting AutoUpdate...
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144614 sha256=7707ec8c06f730f8ab539d524a4c67340c9be667a9b858840adf9127898f49a7
  Stored in directory: C:\Users\SSROCKS\AppData\Local\Temp\pip-ephem-wheel-cache-ihmyx7lz\wheels\1f\be\48\13754633f1d08d1fbfc60d5e80ae1e5d7329500477685286cd
Successfully built antlr4-python3-runtime
   ---------------------------------------- 2/2 [omegaconf]

requirements: A

Finding representations: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:28<00:00,  1.51s/it]


25-05-23 13:35:38 - There are now 19 representations in ds_model_facenet512_detector_opencv_aligned_normalization_base_expand_0.pkl
25-05-23 13:35:38 - Searching ./faces/face0.jpg in 19 length datastore
25-05-23 13:35:38 - find function duration 28.972068548202515 seconds
25-05-23 13:35:38 - Searching ./faces/face1.jpg in 19 length datastore
25-05-23 13:35:39 - find function duration 0.18999719619750977 seconds
25-05-23 13:35:39 - Searching ./faces/face2.jpg in 19 length datastore
25-05-23 13:35:39 - find function duration 0.2025153636932373 seconds
25-05-23 13:35:39 - Searching ./faces/face3.jpg in 19 length datastore
25-05-23 13:35:39 - find function duration 0.193131685256958 seconds
25-05-23 13:35:39 - Searching ./faces/face4.jpg in 19 length datastore
25-05-23 13:35:39 - find function duration 0.19100022315979004 seconds
['Cyril_Ramaphosa', 'Lula_da_Silva', 'Ajay_Banga', 'Joe_Biden', 'Narendra_Modi']
